In [0]:
#!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance 

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))

  # {'key': 'Gender', 'value': ['Men']}
  for item in features: 
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value 

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [0]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )
len(keys)

476

In [0]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_designed for', 'feat_watch case shape', 'feat_hooded',
       'feat_antiscratch lens coating', 'feat_number of heat settings',
       'feat_protects against', 'feat_face shape', 'feat_issued/ not-issued',
       'feat_profession', 'feat_waist size'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False ==  df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [0]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]


df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
   

In [0]:
df[ 'brand'] = df[ 'brand'].map( lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'],model)

(-57.31197897626437, 4.164434984375346)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_clothing category',
 'feat_location - country',
 'feat_certifications and listings',
 'feat_catalog',
 'feat_fabrication',
 'feat_multi pack indicator',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_item character_cat',
 'feat_lens technology_cat',
 'feat_best sellers rank_cat',
 'feat_boot height_cat',
 'feat_case tone_cat',
 'feat_band type_cat',
 'feat_batteries required_cat',
 'feat_licensed_cat',
 'feat_military style_cat',
 'feat_instrument_cat',
 'feat_fits models_cat',
 'feat_chain length_cat',
 'feat_item package quantity_cat',
 'feat_primary shelf id_cat',
 'feat_safety features_cat',
 'feat_theme_cat',
 'feat_fishing_cat',
 'feat_compatible devices_cat',
 'feat_uv rating_cat',
 'feat_metal color_cat',
 'feat_diamond color_cat',
 'feat_date fi

In [0]:
feats = [ 'brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat' ]
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats,model)

In [0]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.18411486416265, 4.196204976054177)


Weight,Feature
0.2531 ± 0.0076,brand_cat
0.1059 ± 0.0078,feat_material_cat
0.0261 ± 0.0029,feat_gender_cat
0.0182 ± 0.0016,feat_brand_cat
0.0119 ± 0.0009,feat_shape_cat
0.0092 ± 0.0015,feat_metal type_cat
0.0033 ± 0.0013,feat_style_cat
0.0001 ± 0.0000,feat_sport_cat


In [0]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'gender': 'men', 'shoe category': "men's shoes", 'color': 'cargo khaki/sail/sequoia/bamboo', 'model': 'nikm-hyperdunk2015-cargokha432.10m', 'manufacturer part number': 'nikm-hyperdunk2015-cargokha432.10m', 'brand': 'nike'},
       {'material': 'synthetic', 'gender': 'men', 'size': '8.5', 'color': 'purple', 'model': '620465 501', 'manufacturer part number': '620465 501', 'brand': 'nike', 'age group': 'adult'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'calypso/white', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '718552410', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'sport': 'casual shoes', 'occasion': 'casual shoes', 'material': 'flyknit', 'gender': 'men', 'shoe size': '10 us men', 'size': '10', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'flyknit', 'color': 'black/lsr orng/grnd prpl/n trq', 'model': '554969 085', 'shoe closure':

In [123]:
!git add matrix_one/day5.ipynb

fatal: pathspec 'matrix_one/day5.ipynb' did not match any files
